# Error video

In [1]:
import sys
sys.path.append('/'.join(sys.path[0].split('/')[:-1]))

import os
import subprocess
import glob
import time

import xarray as xr
import numpy as np
import healpy as hp
from scipy import interpolate

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy

from modules.plotting import plot_signal
from modules.utils import load_test_data
from modules.test import compute_rmse_healpix, compute_relBIAS, compute_rSD, compute_R2
from modules.data import hp_to_equiangular

DATADIR = '../data/healpix/'
lead_time = 6
max_lead_time = 5*24
nodes = 3072
resolution = 5.625

model_description = 'all_const_len2_delta6'

obs = xr.open_mfdataset(DATADIR + 'predictions/observations.nc', combine='by_coords')
pred = xr.open_mfdataset(DATADIR + 'predictions/spherical_unet_' + model_description + '.nc', combine='by_coords')

In [22]:
def create_video(error_ds, var, var_name, folder, lead_time=6, resolution=5.625):
    
    proj = ccrs.PlateCarree()
    lead_times = np.arange(lead_time, max_lead_time+lead_time, lead_time)
    
    vmin_ = error_ds[var].min(xr.ALL_DIMS).values
    vmax_ = error_ds[var].max(xr.ALL_DIMS).values
    
    vmin = min(vmin_, -vmax_)
    vmax = max(vmax_, -vmin_)
    
    for lead_time_idx, lead in enumerate(lead_times):
    
        f, ax = plt.subplots(1, 1, figsize=(10, 5), subplot_kw=dict(projection=proj))

        sample_in = hp_to_equiangular(error_ds.isel(lead_time=lead_time_idx), resolution)
        plot_signal(f, sample=sample_in, var=var, vmin=vmin, vmax=vmax, proj=proj, ax=ax, cbar_shrink=0.8)
        ax.set_title(var_name + ' (forecast - observation), forecast time = {} h'.format(lead), fontsize=15)

        f.tight_layout(pad=-2);
        filename = 'error_' + var + '_' + str(lead_time_idx) + ".png"
        plt.savefig(folder + filename, bbox_inches = 'tight')

    
    subprocess.call([
        'ffmpeg',  '-r', '2', '-f', 'image2', '-s', '1920x1080', '-i', folder+'error_' + var + '_%d.png', '-vcodec',
        'libx264', '-crf', '25', '-pix_fmt', 'yuv420p', folder+'error_'+var+'.mp4'
    ])
    
    for file_name in glob.glob(folder + "error_" + var + "*.png"):
        os.remove(file_name)

In [25]:
%%capture

time_sample = 12
error = (pred.isel(time=time_sample) - obs.isel(time=time_sample)).load()

create_video(error, 'z', 'Z500', DATADIR + 'videos/')
create_video(error, 't', 'T850', DATADIR + 'videos/')